In [1]:
import pandas as pd

In [28]:
# Load data into a pandas DataFrame
df = pd.read_csv('data1-fund_nav.csv')

In [30]:
# Convert date strings to datetime objects
df['nav_date'] = pd.to_datetime(df['nav_date'], format='%Y%m%d')
df.head()

,nav_date,fund_code,amc_code,nav
0,2020-09-25,PWIN,PAMC,15.9151
1,2020-09-28,PRINCIPAL VNEQ-A,PRINCIPAL,8.2898
2,2020-09-28,TLDIVRMF,TALISAM,7.4220
3,2020-09-25,TMBGPROP,TMBAM,8.0741
4,2020-09-25,GC,UOBAM,17.4287


In [31]:
# Calculate daily returns
df['daily_return'] = df.groupby('fund_code')['nav'].pct_change()

In [32]:
# Calculate monthly returns and reset the index

monthly_returns = df.groupby(['fund_code', pd.Grouper(key='nav_date', freq='MS')])['nav'].agg(lambda x: x.iloc[-1] / x.iloc[0] - 1).reset_index()
monthly_returns.rename(columns={'nav': 'monthly_return'}, inplace=True)


In [33]:
# Calculate annualized daily returns
annualized_daily_returns = df.groupby(['fund_code', pd.Grouper(key='nav_date', freq='Y')])['daily_return'].agg(lambda x: (1 + x.mean()) ** 252 - 1).reset_index()
annualized_daily_returns.rename(columns={'daily_return': 'annualized_daily_return'}, inplace=True)
annualized_daily_returns['annualized_daily_return'] = annualized_daily_returns['annualized_daily_return'] * 100



In [34]:
# Calculate annualized monthly returns
annualized_monthly_returns = monthly_returns.groupby(['fund_code', pd.Grouper(key='nav_date', freq='Y')])['monthly_return'].agg(lambda x: (1 + x.mean()) ** 12 - 1).reset_index()
annualized_monthly_returns.rename(columns={'monthly_return': 'annualized_monthly_return'}, inplace=True)
annualized_monthly_returns['annualized_monthly_return'] = annualized_monthly_returns['annualized_monthly_return'] * 100


In [37]:
# Merge annualized returns into a single DataFrame
returns = pd.merge(annualized_daily_returns, annualized_monthly_returns, on=['nav_date','fund_code'], how='outer')
returns = returns.sort_values(['fund_code', 'nav_date'])

In [38]:
# Output results
print(returns)

           fund_code   nav_date  annualized_daily_return  \
0                 GC 2020-12-31                 7.567401   
1                 GC 2021-12-31                 1.111412   
2                 GC 2022-12-31                -8.018249   
3                 GC 2023-12-31                 3.854291   
4   PRINCIPAL VNEQ-A 2020-12-31                11.662638   
5   PRINCIPAL VNEQ-A 2021-12-31                 8.182440   
6   PRINCIPAL VNEQ-A 2022-12-31                -5.992248   
7   PRINCIPAL VNEQ-A 2023-12-31                -0.098279   
8               PWIN 2020-12-31                16.357447   
9               PWIN 2021-12-31                 2.096523   
10              PWIN 2022-12-31                -4.849854   
11              PWIN 2023-12-31                 5.319035   
12          TLDIVRMF 2020-12-31                 6.937904   
13          TLDIVRMF 2021-12-31                 4.980561   
14          TLDIVRMF 2022-12-31                -0.588291   
15          TLDIVRMF 2023-12-31         